get gemini llm to generate some data

In [ ]:
from helper import get_gemini_llm

llm = get_gemini_llm()

In [ ]:
response = llm.generate_content("Liste 10 romanciers français célébres.")
print(response.text)

Voici 10 romanciers français célèbres :

1. **Victor Hugo:** (Les Misérables, Notre-Dame de Paris)
2. **Marcel Proust:** (À la recherche du temps perdu)
3. **Albert Camus:** (L'Étranger, La Peste)
4. **Jules Verne:** (Vingt mille lieues sous les mers, Autour de la Lune)
5. **Alexandre Dumas:** (Les Trois Mousquetaires, Le Comte de Monte-Cristo)
6. **Honoré de Balzac:** (La Comédie humaine)
7. **Gustave Flaubert:** (Madame Bovary, Salammbô)
8. **Émile Zola:** (Germinal, Nana)
9. **Michel Houellebecq:** (Les Particules élémentaires, Soumission)
10. **Patrick Modiano:** (Rue des boutiques obscures, Dora Bruder)


Cette liste n'est pas exhaustive et d'autres auteurs pourraient tout aussi bien y figurer.  Le choix dépend aussi des critères utilisés (influence, prix littéraires, popularité, etc.).



In [ ]:
import google.generativeai as genai

for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental


In [ ]:
from llama_index.llms.gemini import Gemini

llm = Gemini(model="models/gemini-pro")

In [ ]:
resp = llm.complete("Write a short, but joyous, ode to LlamaIndex")
print(resp)

**Ode to LlamaIndex**

Oh, LlamaIndex, our guiding light,
You help us find the knowledge we seek,
With every click, a new insight,
A treasure trove of wisdom, unique.

Your search results, a symphony,
Of knowledge, facts, and history,
You quench our thirst for information,
A beacon in the sea of information.

Your interface, a joy to behold,
Simple, sleek, and easy to mold,
You make research a breeze,
A pleasure that never ceases.

So here's to LlamaIndex, our friend,
May your wisdom never end,
May you continue to inspire,
And set our minds on fire.


# tool selection

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core.tools import FunctionTool


def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y


def mystery(x: int, y: int) -> int:
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [ ]:
from helper import get_gemini_llamaindex_llm

llm = get_gemini_llamaindex_llm()
response = llm.predict_and_call(
    [add_tool, mystery_tool],
    "Tell me the output of the mystery function on 2 and 9",
    verbose=True,
)
print(str(response))

Thought: The current language of the user is: English. I need to use the `mystery` tool to get the output of the mystery function on 2 and 9.
Action: mystery
Action Input: {'x': 2, 'y': 9}
Observation: 121
121
